# Volume 3: Recurrent Neural Networks    
    <Name>
    <Class>
    <Date>

In [ ]:
!pip install --upgrade music21
from music21 import converter, instrument, note, chord, stream
import glob
import os
import gzip
import tarfile
from torchvision import datasets
import numpy as np
import torch
import torch.nn as nn
import torch.optim
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from tqdm import tqdm

assert torch.cuda.is_available()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# Import Drive for easier saving
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def download_data(filepath):
    if not os.path.exists(os.path.join(filepath, 'mozart_sonatas.tar.gz')):
        datasets.utils.download_url('https://github.com/Foundations-of-Applied-Mathematics/Data/raw/master/RNN/mozart_sonatas.tar.gz', filepath, 'mozart_sonatas.tar.gz', None)

    print('Extracting {}'.format('mozart_sonatas.tar.gz'))
    gzip_path = os.path.join(filepath, 'mozart_sonatas.tar.gz')
    with open(gzip_path.replace('.gz', ''), 'wb') as out_f, gzip.GzipFile(gzip_path) as zip_f:
        out_f.write(zip_f.read())

    print('Untarring {}'.format('mozart_sonatas.tar'))
    tar_path = os.path.join(filepath, 'mozart_sonatas.tar')
    z = tarfile.TarFile(tar_path)
    z.extractall(tar_path.replace('.tar', ''))


# Problem 1 #

Download the data.
Write a function that accepts the path to the .mid files, parses the files, and returns a list of the 114215 Notes and Chords as strings. 
There are many element types in MIDI files, so be sure to only look for Notes and Chords.
For the Chords, join the pitches of the Notes in the Chords with a . as in (D3.D2).

Print the length of your list and the number of unique Notes and Chords.


# Problem 2 #

Using the list returned in Problem 1, create the training, validation, and testing DataLoaders.
Make sure to do all of the following steps:


1) Convert the pitches to integers.

2) Split the data into Long tensors of length 10.

3) Create the labels.

4) Randomly split the data into training, validation, and test sets using an 70/15/15 split (use torch.utils.data.random(data,lengths) where lengths=[0.7, 0.15, 0.15]).

5) Create the DataLoaders for these sets of data, using batch_size=128 for the training data and batch_size=32 for the validation and test data; also, set shuffle=True for the training data and False for the validation and test data (this is common practice in Deep Learning).
 

Print the length of each DataLoader (they should be 624, 536, and 536, respectively).

# Problem 3 #

Create an LSTM network class.
Have a hidden layer size of 256, and include at least $3$ LSTM layers.
Also have at least $2$ Linear layers.
The last LSTM layer and each of the Linear layers should be followed by a BatchNorm1d layer except for the last layer, for a total of at least $2$ total BatchNorm layers.

Initialize the model.
Define the loss as CrossEntropyLoss, and define the optimizer as RMSprop.

Train the model for 30 epochs.
Make sure to reinitialize the hidden states (h0, h1) for each training batch.
After taking a backwards step during training, scale the gradients using
nn.utils.clip_grad_norm_(model.parameters(), 5).
This will ensure that the gradients are reasonably sized so that the model can learn.

At the end of every epoch, calculate the validation accuracy and mean loss on the validation data.
Remember to change the model to eval() mode when running the validation data and then train() when running on the training data.
The hidden states (h0, h1) will also need to be reinitialized for each validation batch.

Once the training is complete, plot the training and validation losses versus epochs on the same plot.
Also, plot the validation accuracy versus epochs.
Then, print the final test accuracy by running the finished model on the test data.

In [ ]:
class RNN(nn.Module):
    """ Recurrent Neural Network Class """

    def __init__(self, n_notes, embedding_dim):
        super(RNN, self).__init__()
    
        # initialize layers
      
    def forward(self, x, hidden):
        pass
        # forward pass

    def init_hidden(self, batch_size):
        # initialize the hidden layers
        weight = next(self.parameters()).data
        h0 = weight.new(self.num_layers, batch_size, self.hidden_size).zero_().to(device)
        h1 = weight.new(self.num_layers, batch_size, self.hidden_size).zero_().to(device)
        return (h0, h1)

# Problem 4 #

Write a function that randomly chooses a sequence in the test data (which has length 10) and predicts the next $n$ elements, defaulting to 500.
Convert the predicted elements to pitches, and return them as a list of length $n$.

In [ ]:
def load_model(filename):
    """ Load a saved model to continue training or evaluate """
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # n_notes is the number of unique pitches
    model = Network(n_notes, 32)
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters(), lr=.001)

    checkpoint = torch.load(filename, map_location=torch.device('cpu'))
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    last_epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    model.eval() # Toggle evaluation mode

    return model, criterion, optimizer

# Problem 5 # 

Convert the predictions from Problem 4 into Music21 Note and Chord objects and save it as 'mozart.mid'. Embed your music file into the notebook.